# Adding Conditional Control to Text-to-Image Diffusion Models

참고링크
+ blog
    + https://kimjy99.github.io/%EB%85%BC%EB%AC%B8%EB%A6%AC%EB%B7%B0/controlnet/
    + https://junia3.github.io/blog/controlnet
+ paper : https://arxiv.org/abs/2302.05543

## Contributions
stable diffusion이라는 large model을 다양한 task에 아주 쉽게(적은 데이터셋 + 적은 컴퓨팅자원) fine tunning할 수 있게 하였다.

## 기존 condition control 방법

<img src="https://drive.google.com/uc?id=130yyc3D_P9OCcMu4YEtCBWM1w6w2v3Bt" height=300>

원하는 condition의 데이터 셋을 통해서, 모델을 학습해주며, __해당 vector를 embedding할 수 있는 CLIP layer를 학습시키고__, 이를 통해 만들어진 condition vector를 생성과정에 cross attention 해주어 condition 제어를 하였다.  
따라서, 원하는 task에 해당하는 conditional encoder만 잘 준비해주면, 제어가 가능하다.

### 문제점
1. 원하는 task에 맞게 모델 바꾸기 위해서 많은 데이터 셋이 필요하다.(기존 stable diffusion의 경우, 56억장의 데이터 셋으로 학습됨)
2. 학습하고난 이후, 모델의 기존 성능을 유지해줘야 한다.(continual learning 문제; 오버피팅, 파괴적 망각현상)
3. diffusion 모델이 비록 수많은 데이터셋으로 학습되었더라도, 모든 task에 대해서 좋은 성능을 보일 수 없다. 즉, 원하는 task에 어울리는 end-to-end 학습을 진행해야 한다.(기존의 방식은 encoder를 바꾸거나, attention을 변주하는 식이었다.)

## ControlNet

### zero convolution이란
__모든 가중치를 0으로 초기화해놓은 1x1 conv layer를 말한다.__


1. 사전학습된 모델을 locked해놓고, 이를 카피한 모델을 가져온다.  
2. copy한 모델의 시작과 끝에 zero convolution을 넣고 변형을 준다.
3. (기존모델 결과값), (condition, copy모델 결과값)을 합하여 최종 결과물 벡터를 생성한다.

Base ControlNet을 통해 어떻게 구성되는지 살펴보면,

<img src="https://drive.google.com/uc?id=1kK3L_dI_aV2Q8oV-EQURAfiR1M0c41Q7" height=220>

기존 네트워크를  
$y = \mathcal{F}(x ; \Theta)$ (1)   
라고 하자.


trainable copy의 파라미터를 $\Theta_c$라 두고,   
각 zero conv layer를 $\mathcal{Z}(\cdot ; \Theta_{z_i})$라고하면, 위의 그림의 결과를 다음과 같이 수식으로 한번에 표현가능하다.

$y = \mathcal{F}(x ; \Theta) + \mathcal{Z}(\mathcal{F}(x + \mathcal{Z}(c ; \Theta_{z_1}) ; \Theta_c); \Theta_{z_2})$ (2)  


위의 식에서, 학습 처음에 $\mathcal{Z(x)} = 0$이 되도록, 구성했으므로 output은 $\mathcal{F}(x;\Theta)$이다!  
(즉, $y_c = y$)

__zero conv layer의 효과__
+ 기존 Network의 성능을 해치지 않는다.
+ 처음 훈련시, 초깃값에 따른 성능 문제가 아예 없다.(일정한 finetune 성능 기대가능)

__Q. 모든 가중치를 0으로 초기화해놓으면, update할 때, 모든 값이 동일하게 변화되어서 문제 생기지 않나요?__  
A. 만약, zero layer가 연속으로 이어지면 문제가 생길 수 있지만, 중간에 값이 흐르는 layer가 끼워져있어서 첫번째 훈련 step이후로는 정상적인 conv layer로 갱신된다.  
(단, 이전 layer의 결과값이 0 아니라는 가정을 두고, 연쇄법칙으로 계산해보면 확인해볼 수 있다.)

ResNet의 skip connection을 생각해보면, 거기서도 bottleneck에 해당하는 layer의 마지막 output값은 0이 되도록 초기화해준다.

### ControlNet의 장점


+ 추가로 더해지는 학습 파라미터 : ControlNet, 원하는 task에 해당하는 context encoder   
    + 컴퓨팅 자원이 많이 필요하지 않다!
    + (실제로 단일 RTX 3090Ti로 기존의 커다란 산업 모델급 성능을 보였다.)
+ 학습 데이터 : 수십만장의 데이터로 충분히 성능을 보였다.  

### Stable Diffusion에의 적용
<img src="https://drive.google.com/uc?id=1V41D2bZjDAzZroEF8ZsOrmcG4jk62F0E" width=400>

기존 U-Net은 총 25개의 block들로 구성되어있으며,
각각 encoder/decoder block 12개, middle block 1개이며  
각 block에는 4개의 resnet layer와 2개의 VIT가 들어있다.

## Training & Inference

### Train


$L = \mathbb{E}_{z_0, t, c_t, c_f, \epsilon \sim \mathcal{N}(0,1)}\big[||\epsilon - \epsilon_\theta(z_t, t, c_t, c_f)||^2_2\big]$

이 때, $c_t$의 random한 50%는 empty string을 이용했다
+ ControlNet이 $c_f$에 대한 semantic한 정보를 직접적으로 인식하는 능력을 키워주기 위함.
+ 즉, edge, pose, depth등의 의미를 잘 이해시키도록 다른 condition을 제거해줌.

장점
+ __ControlNet과 동시에 end-to-end로 finetuning이 가능하다__
+ 훈련하면서 중간중간에도 높은 퀄리티의 이미지를 생성할 수 있다.
    + zero conv가 기존 net의 noise에 직접적인 영향을 주지 않기 때문(값에 막 더하고 그러지 않고 noise 산출하는 파라미터 자체로 들어감)

따라서, 1만장정도의 step을 거치면서 control net이 갑자기 조건을 갖추는 "sudden convergence 현상"을 관측하였다.

<img src="https://drive.google.com/uc?id=1qZiYjb0hwMQfHtuzHS8VfL4aiyv3hvRS" height=300>

### Inference

stable diffusion은 Classifier-Free-Guidance(CFG)기술을 통해 condition image를 생성해냈다.  
$\epsilon_{prd} = \epsilon_{uc} + \beta_{cfg}(\epsilon_c - \epsilon_{uc})$

여기에, ControlNet을 추가하기 위해 다음과 같은 가중치를 추가로 도입한다.

i번째 block에 대해서, $w_i = 64/h_i$값을 Diffusion과 ControlNet사이에 연결값에 곱해준다!

## Experiments



<img src="https://drive.google.com/uc?id=1_NsFwGRpdh01FTMjAU34-1GNjizqE7Ip">

### Ablative study
<img src="https://drive.google.com/uc?id=1wnFzTSNbo51gznXgS_7WKSTWK8IpJgj0">

왼쪽부터 prompt를 no / 불충분 / 아예다른 / 완벽 순이다.  
zero conv를 사용했을 때 가장 퀄리티가 좋음을 알 수 있다.

<img src="https://drive.google.com/uc?id=1bjkg3oSdKfok5kCxMZqc53B9z3tQJILA" height=200>